<a href="https://colab.research.google.com/github/Jamnic98/blue-cheese-classifier/blob/main/blue_cheese_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# mount the google drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
import numpy as np
import shutil
import random

class_names = ['blueCheese', 'notBlueCheese']
root_dir = '/content/drive/MyDrive/'
val_ratio = 0.15
test_ratio = 0.05

for class_name in class_names:
    os.makedirs(root_dir +'train/' + class_name)
    os.makedirs(root_dir +'val/' + class_name)
    os.makedirs(root_dir +'test/' + class_name)

    cheeses = os.listdir(root_dir + class_name)
    # iterate through cheese folders then copy each image into the appropriate 
    # class folder
    for cheese_name in cheeses:
      cheese_dir = root_dir + class_name + '/' + cheese_name
      cheese_images = os.listdir(cheese_dir)
      
      ### TODO: change shuffle location
      np.random.shuffle(cheese_images)

      train_FileNames, val_FileNames, test_FileNames = np.split(
        np.array(cheese_images),
        [int(len(cheese_images)* (1 - (val_ratio + test_ratio))), 
         int(len(cheese_images)* (1 - test_ratio))]
      )

      train_FileNames = [cheese_dir + '/'+ name for name in train_FileNames.tolist()]
      val_FileNames = [cheese_dir + '/' + name for name in val_FileNames.tolist()]
      test_FileNames = [cheese_dir + '/' + name for name in test_FileNames.tolist()]

      # Copy-pasting images
      for train_FileName in train_FileNames:
          shutil.copy(train_FileName, root_dir +'train/' + class_name)

      for val_FileName in val_FileNames:
          shutil.copy(val_FileName, root_dir +'val/' + class_name)

      for test_FileName in test_FileNames:
          shutil.copy(test_FileName, root_dir +'test/' + class_name)


    

KeyboardInterrupt: ignored